In [ ]:
1

In [ ]:


import datetime as dt
from pathlib import Path
import duckdb
import plotly.express as px


BASE_DIR = Path().resolve()
DATA_DIR = BASE_DIR / "data"
DATABASE_DIR = DATA_DIR / "database"

AIR_QUALITY_DB_PATH = DATABASE_DIR / "air_quality_db.db"
SCHEMA = "raw"
TABLE = "air_quality_raw"
VIEW_NAME = "air_quality_df"
DATABASE_DIR.mkdir(parents=True, exist_ok=True)
        


In [ ]:
import plotly.express as px

query = f"SELECT time, pm2_5, pm10 FROM {SCHEMA}.{TABLE} ORDER BY time"
with duckdb.connect(database=str(AIR_QUALITY_DB_PATH)) as con:
    df = con.sql(query).df()
if df.empty:
    raise ValueError("No data available to plot. Load data first.")

fig = px.line(
    df,
    x="time",
    y=["pm2_5", "pm10"],
    template="plotly_dark",   # dark mode base
    color_discrete_map={
        "pm2_5": "#a19090",   # warm bright red
        "pm10":  "#4da0e3",   # cool bright blue
    },
)

# make the lines smoother + slightly thicker
fig.update_traces(mode="lines", line=dict(width=2))

# styling
fig.update_layout(
    title="How Dusty Is the Air in Paris? PM2.5 and PM10 particules Over Time",
    yaxis_title="µg/m³",
    legend=dict(title=None, orientation="h", y=-0.25),
    margin=dict(t=60, l=60, r=20, b=60),
    plot_bgcolor="#DDD0D0",
    paper_bgcolor="#f6f2f2",
    font=dict(color="#100f0f"),
)

fig.add_hline(
    y=15,
    line_color="#774D4D",   # pure white (hex)
    line_dash="dot",
    annotation_text="WHO PM2.5 (15)",
    annotation_position="top left",
    annotation_font_color="#060606",
)
fig.update_xaxes(showgrid=False, zeroline=False)
fig.update_yaxes(showgrid=False, zeroline=False)

fig.add_hline(
    y=45,
    line_color="#764848",   # light grey
    line_dash="dot",
    annotation_text="WHO PM10 (45)",
    annotation_position="bottom left",
    annotation_font_color="#0D0D0D",
)

fig.show()
